# Multi-head classification with AutoKeras

## Input
Functional connectivity of the size N_networks * N_networks. The connectivity matrix is symmetric so it will be flattened and only upper traingle (k=1) will be kept.

## Output
There are two outputs:
1. Participant rating of their hypnotic depth (1 = hypnotized OR 0 = non-hypnotized)
2. participants hypnotizability score (low OR high)

In [1]:
import pandas as pd
import tensorflow as tf
import autokeras as ak

In [2]:
DATA = pd.read_csv('data/connectivities_yeo17nets.csv', index_col=0)

# keep the blocks where procedure is described as hypnosis
DATA = DATA.query('description == "hypnosis"')

# add the hypnotizability data
hypnotizability = pd.read_excel('data/PLB_HYP_data_MASTER.xlsx', index_col=0)
hypnotizability = hypnotizability['hypnotizability_total'].iloc[9:59]
DATA['hypnotizability_total'] = hypnotizability.repeat(4).reset_index(drop=True)
# drop rows with nan values
DATA = DATA.dropna(axis=0, how='any').reset_index(drop=True)

X = DATA.drop(columns=['hypnosis_depth', 'procedure', 'description', 'session', 'condition',
                       'bids_id', 'hypnotizability_total'])


y_depth = DATA['hypnosis_depth'].apply(lambda x: 1 if x > 5 else 0)
y_score = DATA['hypnotizability_total'].apply(lambda x: 'high' if x > 5 else 'low')
y_depth.value_counts(normalize=True), y_score.value_counts(normalize=True)

(1    0.560976
 0    0.439024
 Name: hypnosis_depth, dtype: float64,
 high    0.609756
 low     0.390244
 Name: hypnotizability_total, dtype: float64)

In [7]:
clf = ak.StructuredDataClassifier(max_trials=10, project_name='temp/structured_data_classifier')
history = clf.fit(X, y_depth)
clf.evaluate(X, y_depth)

INFO:tensorflow:Reloading Oracle from existing project ./temp/structured_data_classifier/oracle.json
INFO:tensorflow:Reloading Tuner from ./temp/structured_data_classifier/tuner0.json


2022-05-26 13:21:28.261502: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-05-26 13:21:28.695971: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-05-26 13:21:28.699676: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-05-26 13:21:28.702329: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-05-26 13:21:28.705505: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-05-26 13:21:28.709138: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-05-26 13:21:28.713057: I tensorflow/core/grappler/optimizers/cust

INFO:tensorflow:Oracle triggered exit


2022-05-26 13:21:30.560992: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-05-26 13:21:30.618336: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-05-26 13:21:30.665760: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-05-26 13:21:30.715640: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-05-26 13:21:30.762015: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-05-26 13:21:30.813053: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-05-26 13:21:30.852579: I tensorflow/core/grappler/optimizers/cust

3/3 [==============================] - 15s 3s/step - loss: 0.8108 - accuracy: 0.4878
INFO:tensorflow:Assets written to: ./temp/structured_data_classifier/best_model/assets


2022-05-26 13:22:10.352997: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-05-26 13:22:10.359172: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-05-26 13:22:10.365750: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-05-26 13:22:10.371094: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-05-26 13:22:10.377584: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-05-26 13:22:10.385076: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-05-26 13:22:10.393276: I tensorflow/core/grappler/optimizers/cust

3/3 [==============================] - 16s 3s/step - loss: 0.6121 - accuracy: 0.7561


[0.6121454238891602, 0.7560976147651672]